#                       Перевірка розподілу мандатів на місцевих виборах

На сайті ЦВК знаходимо посилання на сторінку з результатами виборів до районної/міської/обласної ради.

Зразок лінку - http://www.cvk.gov.ua/pls/vm2015/PVM057?PID112=30&PID102=9113&PF7691=9113&PT001F01=100&rej=0&pt00_t001f01=100

In [15]:
# Вставляємо лінк сюди
url = 'http://www.cvk.gov.ua/pls/vm2015/PVM057?PID112=30&PID102=9113&PF7691=9113&PT001F01=100&rej=0&pt00_t001f01=100' 

In [11]:
import pandas as pd
# Імпортуємо таблицю з результатами з сайту ЦВК
df = pd.read_html(url)[3]
df = df.drop(df.index[0])
df.columns = ['party','votes','mandates','perc']
df[['votes','mandates']] = df[['votes','mandates']].apply(pd.to_numeric)
# Підраховуємо відсоток голосів кожної партії
df['perc'] = df.votes / df.votes.sum() * 100
# Визначаємо партії що подолали 5-ти відсотковий бар'єр та підраховуємо виборчу квоту.
df = df[df.perc>=5]
df1 = pd.read_html(url)[2]
df1.columns = ['raz','dva']
kilk_dep = df1.dva.iloc[0]
print('Загальна кількість мандатів у раді - ' +str(kilk_dep))
kvota = df.votes.sum() / kilk_dep
print('Виборча квота - ' + str(kvota))
df['mand'] = df.votes/kvota
# Кількість мандатів за розподілена за цілими числами
df['mand_r']= round(df.mand-0.5,0)
# Підраховуємо дробові залишки
df['chast'] = df.mand - df.mand_r
# Визначаємо кількість нерозподілених мандатів
rizn_mand = kilk_dep - df.mand_r.sum()
df = df.sort_values('chast',ascending=False)
# Фінальний розподіл без дробових залишків
df['mand_final'] = df.mand_r + 0
df = df.reset_index(drop=True)
# Розподіл мандатів за дробовими залишками
for i in range(0,int(rizn_mand)):
    df.at[i, 'mand_final'] = df.mand_r.iloc[i] + 1
# Різниця між кількістю мандатів між даними ЦВК та нашим підрахунком
df['rizn'] = df['mandates'] -df['mand_final']
# Перевірка на помилки
ac = df[df['rizn' ] !=0]
if len(ac.index)> 0:
    print('Вкрадені мандати')
    # Визначаємо хто отримав та втратив мандат
    df1 = ac[ac.rizn>0]
    dovj = len(df1.index)
    parties = []
    while dovj > 0:
        parties.append(df1.party.iloc[dovj-1])
        dovj -= 1
    print('Партії що отримали зайві мандати -' + str(parties))
    df1 = ac[ac.rizn<0]
    dovj = len(df1.index)
    partiesl = []
    while dovj > 0:
        partiesl.append(df1.party.iloc[dovj-1])
        dovj -= 1
    print('Партії що втратили мандати - ' + str(partiesl))
else:
    print('Помилок не виявлено') 

Загальна кількість мандатів у раді - 36
Виборча квота - 645.25
Партії що отримали зайві мандати -['ПАРТІЯ "БЛОК ПЕТРА ПОРОШЕНКА "СОЛІДАРНІСТЬ"']
Партії що втратили мандати - ['Партія "Відродження"']


Таблиця розподілу мандатів

In [14]:
df.sort_values('mand_final',ascending=False)

,party,votes,mandates,perc,mand,mand_r,chast,mand_final,rizn
5,"ПАРТІЯ ""БЛОК ПЕТРА ПОРОШЕНКА ""СОЛІДАРНІСТЬ""",4851,8,18.421053,7.518016,7.0,0.518016,7.0,1.0
1,"політична партія Всеукраїнське об’єднання ""Бат...",3817,6,14.494570,5.915537,5.0,0.915537,6.0,0.0
6,"політична партія Всеукраїнське об’єднання ""Сво...",4141,6,15.724918,6.417668,6.0,0.417668,6.0,0.0
0,"Політична партія ""Радикальна партія Олега Ляшк...",1926,3,7.313739,2.984890,2.0,0.984890,3.0,0.0
2,"ПОЛІТИЧНА ПАРТІЯ ""УКРАЇНСЬКЕ ОБ’ЄДНАННЯ ПАТРІО...",1808,3,6.865649,2.802015,2.0,0.802015,3.0,0.0
3,"Політична партія ""Рідне місто""",1662,3,6.311233,2.575746,2.0,0.575746,3.0,0.0
4,"Партія ""Відродження""",1625,2,6.170730,2.518404,2.0,0.518404,3.0,-1.0
7,"ПОЛІТИЧНА ПАРТІЯ ""ЗА КОНКРЕТНІ СПРАВИ""",2080,3,7.898534,3.223557,3.0,0.223557,3.0,0.0
8,Народна Партія,1319,2,5.008734,2.044169,2.0,0.044169,2.0,0.0
